In [1]:
from jax import random, numpy as jnp
from stadion.models import LinearSDE, MLPSDE
from pprint import pprint
from stadion import kds_loss
import numpy as onp
import ot

2025-02-08 11:20:25.624270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739031625.637669 2189536 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739031625.641708 2189536 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from nn import NNSDE
from nonlinear_script import *

In [3]:
n = 2000
d = 20
r = 2
n_envs = 10
n_test_envs = 10
epsilon = 1.5
gamma = 0.98
activation = "linear"
model_activation = "learned"

model_hidden_size = 8

bandwidth = 7.0
steps = 30000
estimator = "linear"
reg = 0.0001

intv_scale = 0.05

n_samples_burnin=200

seed = 50

In [4]:
### Mini
# d = 2
# r = 2
# n_envs = 2
# n_test_envs = 2
# steps=30000
# intv_scale = 0.05
# bandwidth=1.0
# bandwidth=3.0

In [5]:
key = random.PRNGKey(seed)

true_model = build_model(r, activation, epsilon, gamma, n_samples_burnin)
key, true_model, param = initialize_model(key, true_model, d)

# key, subk1, subk2 = random.split(key, 3)
# param["mlp_0"] = sample_stiefel(subk1, d, r)
# param["mlp_1"] = sample_stiefel(subk2, d, r).T

key, intv_param, datasets, targets = build_data(key, true_model, d, n_envs, intv_scale, n)
key, test_intv_param, test_datasets, test_targets = build_data(key, true_model, d, n_test_envs, intv_scale, n)

model = build_model(model_hidden_size, model_activation, epsilon, gamma, n_samples_burnin)
model = fit_model(model, datasets, targets, intv_param, bandwidth, steps, estimator, reg)

# wds, mean, std = assess_model(key, model, n_test_envs, test_intv_param, test_datasets, n)
wds, mean, std = assess_model_mean(key, model, n_test_envs, test_intv_param, test_datasets, n)
# wds, mean, std = assess_model_cov(key, model, n_test_envs, test_intv_param, test_datasets, n)

print(wds)
print(mean, std)

W0000 00:00:1739031656.509575 2189536 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


step:  3000 kds:     0.011375  | min remain:  1.4  sec/step: 0.003
step:  6000 kds:     0.001846  | min remain:  1.0  sec/step: 0.003
step:  9000 kds:     0.001979  | min remain:  0.9  sec/step: 0.003
step: 12000 kds:     0.001743  | min remain:  0.8  sec/step: 0.003
step: 15000 kds:     0.001459  | min remain:  0.7  sec/step: 0.003
step: 18000 kds:     0.001577  | min remain:  0.5  sec/step: 0.003
step: 21000 kds:     0.001365  | min remain:  0.4  sec/step: 0.003
step: 24000 kds:     0.001280  | min remain:  0.3  sec/step: 0.003
step: 27000 kds:     0.001360  | min remain:  0.1  sec/step: 0.003
step: 30000 kds:     0.001193  | min remain:  0.0  sec/step: 0.003
[1.7197045 3.9534829 4.168993  4.3935533 9.982978  4.0282927 5.2655916
 6.064933  4.5570793 1.8704779]
4.600509 2.197429


In [6]:
#s: 3.5693474 1.5361391
#l: 

In [7]:
Y = jnp.mean(jnp.stack(test_datasets), axis = 1)
X = jnp.mean(Y, axis = 0, keepdims = True)
norms = jnp.linalg.norm(X - Y, axis = 1)
print(norms, jnp.mean(norms), jnp.std(norms))

[3.5165555 1.7679259 1.4905057 1.1689508 7.0334754 3.3718562 2.523505
 3.7070084 1.7505672 3.427846 ] 2.9758198 1.6213742


In [8]:
#oh duh
#test the minimum possible error if you use the true model for the test interventions
wds, mean, std = assess_model_mean(key, true_model, n_test_envs, test_intv_param, test_datasets, n)
print(wds)  
print(mean, std)

[0.9435499  2.6844406  1.255003   1.7170272  1.4116145  1.2211748
 0.99792814 2.2888582  1.2750317  1.8049145 ]
1.5599543 0.5373293


In [9]:
# x = jnp.zeros(d)
# for i in range(100):
#     x = x + true_model.f(x, true_model.param, test_intv_param.index_at(0))
#     if i >= 98:
#         print(x)